In [10]:
import pandas as pd
import plotly.express as px

# ------------------------------------------------------------------
# 1) Load the detailed deaths+demographics dataset
# ------------------------------------------------------------------
df = pd.read_csv("https://raw.githubusercontent.com/Ramil-cyber/Research_Linking_Analyzing_Deaths_US_Prisons/refs/heads/main/Data/Last_merged_full_df.csv", low_memory=False)

# ------------------------------------------------------------------
# 2) Aggregate to one row per state & year
#    - sum death_count
#    - take first value of each demographic/metric (they're constant per state-year)
# ------------------------------------------------------------------
agg = df.groupby(['state_name', 'state_abbr', 'death_year'], as_index=False).agg({
    'death_count': 'sum',
    'total_pop_15to64': 'first',
    'female_pop_15to64': 'first',
    'male_pop_15to64': 'first',
    'aapi_pct': 'first',
    'black_pct': 'first',
    'latinx_pct': 'first',
    'native_pct': 'first',
    'white_pct': 'first',
    'total_incarceration': 'first',
    'total_incarceration_rate': 'first',
    'total_prison_pop': 'first',
    'female_prison_pop': 'first',
    'male_prison_pop': 'first'
})

# ------------------------------------------------------------------
# 3) Build animated choropleth by year
# ------------------------------------------------------------------
fig = px.choropleth(
    agg,
    locations='state_abbr',
    locationmode='USA-states',
    color='death_count',
    animation_frame='death_year',
    scope='usa',
    color_continuous_scale='Reds',
    labels={'death_count': 'Deaths', 'death_year': 'Year'},
    hover_data={
        'state_abbr': False,
        'state_name': True,
        'death_count': True,
        'total_pop_15to64': ':.0f',
        'aapi_pct': ':.1f',
        'black_pct': ':.1f',
        'latinx_pct': ':.1f',
        'native_pct': ':.1f',
        'white_pct': ':.1f',
        'total_incarceration_rate': ':.0f',
        'female_prison_pop': ':.0f',
        'male_prison_pop': ':.0f'
    },
    title='US Prisoner Deaths by State (2015–2019)'
)

# ------------------------------------------------------------------
# 4) Tweak layout
# ------------------------------------------------------------------
fig.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    geo=dict(lakecolor='white')
)

# ------------------------------------------------------------------
# 5) Export & display
# ------------------------------------------------------------------
fig.write_html('prisoner_deaths_state_year_map.html', include_plotlyjs='cdn')
fig.show()


In [11]:
import pandas as pd
import plotly.graph_objects as go

# ------------------------------------------------------------------
# 1) Load and aggregate your data exactly as before
# ------------------------------------------------------------------
df = pd.read_csv(
    "https://raw.githubusercontent.com/Ramil-cyber/Research_Linking_Analyzing_Deaths_US_Prisons/refs/heads/main/Data/Last_merged_full_df.csv",
    low_memory=False,
)
agg = (
    df.groupby(['state_name', 'state_abbr', 'death_year'], as_index=False)
      .agg({
        'death_count': 'sum',
        'total_pop_15to64': 'first',
        'female_pop_15to64': 'first',
        'male_pop_15to64': 'first',
        'aapi_pct': 'first',
        'black_pct': 'first',
        'latinx_pct': 'first',
        'native_pct': 'first',
        'white_pct': 'first',
        'total_incarceration': 'first',
        'total_incarceration_rate': 'first',
        'total_prison_pop': 'first',
        'female_prison_pop': 'first',
        'male_prison_pop': 'first'
      })
)

years = sorted(agg['death_year'].unique())

# State centroid lookup for labels
state_centroids = {
    "AL": {"lat":32.8067,"lon":-86.7911}, "AK": {"lat":61.3707,"lon":-152.4044},
    "AZ": {"lat":33.7298,"lon":-111.4312}, "AR": {"lat":34.9697,"lon":-92.3731},
    "CA": {"lat":36.1162,"lon":-119.6816}, "CO": {"lat":39.0598,"lon":-105.3111},
    "CT": {"lat":41.5978,"lon":-72.7554},  "DE": {"lat":39.3185,"lon":-75.5071},
    "DC": {"lat":38.8974,"lon":-77.0268}, "FL": {"lat":27.7663,"lon":-81.6868},
    "GA": {"lat":33.0406,"lon":-83.6431}, "HI": {"lat":21.0943,"lon":-157.4983},
    "ID": {"lat":44.2405,"lon":-114.4788}, "IL": {"lat":40.3495,"lon":-88.9861},
    "IN": {"lat":39.8494,"lon":-86.2583}, "IA": {"lat":42.0115,"lon":-93.2105},
    "KS": {"lat":38.5266,"lon":-96.7265}, "KY": {"lat":37.6681,"lon":-84.6701},
    "LA": {"lat":31.1695,"lon":-91.8678}, "ME": {"lat":44.6939,"lon":-69.3819},
    "MD": {"lat":39.0639,"lon":-76.8021}, "MA": {"lat":42.2302,"lon":-71.5301},
    "MI": {"lat":43.3266,"lon":-84.5361}, "MN": {"lat":45.6945,"lon":-93.9002},
    "MS": {"lat":32.7416,"lon":-89.6787}, "MO": {"lat":38.4561,"lon":-92.2884},
    "MT": {"lat":46.9219,"lon":-110.4544}, "NE": {"lat":41.1254,"lon":-98.2681},
    "NV": {"lat":38.3135,"lon":-117.0554}, "NH": {"lat":43.4525,"lon":-71.5639},
    "NJ": {"lat":40.2989,"lon":-74.5210}, "NM": {"lat":34.8405,"lon":-106.2485},
    "NY": {"lat":42.1657,"lon":-74.9481}, "NC": {"lat":35.6301,"lon":-79.8064},
    "ND": {"lat":47.5289,"lon":-99.7840}, "OH": {"lat":40.3888,"lon":-82.7649},
    "OK": {"lat":35.5653,"lon":-96.9289}, "OR": {"lat":44.5720,"lon":-122.0709},
    "PA": {"lat":40.5908,"lon":-77.2098}, "RI": {"lat":41.6809,"lon":-71.5118},
    "SC": {"lat":33.8569,"lon":-80.9450}, "SD": {"lat":44.2998,"lon":-99.4388},
    "TN": {"lat":35.7478,"lon":-86.6923}, "TX": {"lat":31.0545,"lon":-97.5635},
    "UT": {"lat":40.1500,"lon":-111.8624}, "VT": {"lat":44.0459,"lon":-72.7107},
    "VA": {"lat":37.7693,"lon":-78.1699}, "WA": {"lat":47.4009,"lon":-121.4905},
    "WV": {"lat":38.4912,"lon":-80.9545}, "WI": {"lat":44.2685,"lon":-89.6165},
    "WY": {"lat":42.7560,"lon":-107.3025}
}

# ------------------------------------------------------------------
# 4) Build the animated figure by hand
# ------------------------------------------------------------------
fig = go.Figure()

for i, yr in enumerate(years):
    dff = agg[agg['death_year']==yr]
    # Choropleth trace
    fig.add_trace(go.Choropleth(
        locations    = dff['state_abbr'],
        locationmode = 'USA-states',
        z            = dff['death_count'],
        zmin         = agg['death_count'].min(),
        zmax         = agg['death_count'].max(),
        colorscale   = 'Reds',
        colorbar     = dict(title='Deaths'),
        hovertemplate=(
            "<b>%{location}</b><br>"
            "Deaths: %{z}<br>"
            "Pop15–64: %{customdata[0]:,}<br>"
            "AAPI%: %{customdata[1]:.1f}<br>"
            "Black%: %{customdata[2]:.1f}<br>"
            "Latinx%: %{customdata[3]:.1f}<br>"
            "Native%: %{customdata[4]:.1f}<br>"
            "White%: %{customdata[5]:.1f}<br>"
            "IncarcRate: %{customdata[6]:.1f}<extra></extra>"
        ),
        customdata   = dff[[
            'total_pop_15to64','aapi_pct','black_pct','latinx_pct',
            'native_pct','white_pct','total_incarceration_rate'
        ]].values,
        visible      = (i==0)
    ))
    # State‐abbr labels
    fig.add_trace(go.Scattergeo(
        lon         = [state_centroids[s]['lon'] for s in dff['state_abbr']],
        lat         = [state_centroids[s]['lat'] for s in dff['state_abbr']],
        text        = dff['state_abbr'],
        mode        = 'text',
        showlegend  = False,
        hoverinfo   = 'none',
        textfont    = dict(size=9, color='black'),
        visible     = (i==0)
    ))

# ------------------------------------------------------------------
# 5) Animation slider
# ------------------------------------------------------------------
steps = []
for i, yr in enumerate(years):
    vis = [False]*(2*len(years))
    vis[2*i]   = True
    vis[2*i+1] = True
    steps.append(dict(
        method='update',
        label=str(yr),
        args=[{'visible': vis},
              {'title':f'US Prisoner Deaths by State — {yr}'}]
    ))

fig.update_layout(
    title=f'US Prisoner Deaths by State — {years[0]}',
    geo=dict(scope='usa', lakecolor='white', bgcolor='#F0F0F0'),
    updatemenus=[dict(
        type    ='buttons',
        showactive=False,
        x       =0.1,
        y       =1.05,
        buttons =[dict(label='Play',
                       method='animate',
                       args=[None, dict(frame=dict(duration=1000, redraw=True),
                                        fromcurrent=True)])]
    )],
    sliders=[dict(
        active     =0,
        pad        =dict(t=50),
        currentvalue=dict(prefix='Year: '),
        steps      =steps
    )],
    margin=dict(l=0, r=0, t=50, b=0)
)

fig.write_html('prisoner_deaths_with_abbr.html', include_plotlyjs='cdn')
fig.show()


In [14]:
import pandas as pd
import plotly.graph_objects as go

# ------------------------------------------------------------------
# 1) Load the merged state-year dataset
# ------------------------------------------------------------------
df = pd.read_csv(
    "https://raw.githubusercontent.com/Ramil-cyber/Research_Linking_Analyzing_Deaths_US_Prisons/refs/heads/main/Data/Last_merged_full_df.csv",
    low_memory=False,
)
# ------------------------------------------------------------------
# 2) Aggregate (sum death_count, take first of other metrics)
# ------------------------------------------------------------------
agg = df.groupby(['state_name', 'state_abbr', 'death_year'], as_index=False).agg({
    'death_count': 'sum',
    'total_pop_15to64': 'first',
    'female_pop_15to64': 'first',
    'male_pop_15to64': 'first',
    'aapi_pct': 'first',
    'black_pct': 'first',
    'latinx_pct': 'first',
    'native_pct': 'first',
    'white_pct': 'first',
    'total_incarceration': 'first',
    'total_incarceration_rate': 'first',
    'total_prison_pop': 'first',
    'female_prison_pop': 'first',
    'male_prison_pop': 'first'
})

# ------------------------------------------------------------------
# 3) State centroids for label placement
# ------------------------------------------------------------------
state_centroids = {
    "AL": {"lat":32.8067,"lon":-86.7911}, "AK": {"lat":61.3707,"lon":-152.4044},
    "AZ": {"lat":33.7298,"lon":-111.4312}, "AR": {"lat":34.9697,"lon":-92.3731},
    "CA": {"lat":36.1162,"lon":-119.6816}, "CO": {"lat":39.0598,"lon":-105.3111},
    "CT": {"lat":41.5978,"lon":-72.7554},  "DE": {"lat":39.3185,"lon":-75.5071},
    "DC": {"lat":38.8974,"lon":-77.0268}, "FL": {"lat":27.7663,"lon":-81.6868},
    "GA": {"lat":33.0406,"lon":-83.6431}, "HI": {"lat":21.0943,"lon":-157.4983},
    "ID": {"lat":44.2405,"lon":-114.4788}, "IL": {"lat":40.3495,"lon":-88.9861},
    "IN": {"lat":39.8494,"lon":-86.2583}, "IA": {"lat":42.0115,"lon":-93.2105},
    "KS": {"lat":38.5266,"lon":-96.7265}, "KY": {"lat":37.6681,"lon":-84.6701},
    "LA": {"lat":31.1695,"lon":-91.8678}, "ME": {"lat":44.6939,"lon":-69.3819},
    "MD": {"lat":39.0639,"lon":-76.8021}, "MA": {"lat":42.2302,"lon":-71.5301},
    "MI": {"lat":43.3266,"lon":-84.5361}, "MN": {"lat":45.6945,"lon":-93.9002},
    "MS": {"lat":32.7416,"lon":-89.6787}, "MO": {"lat":38.4561,"lon":-92.2884},
    "MT": {"lat":46.9219,"lon":-110.4544}, "NE": {"lat":41.1254,"lon":-98.2681},
    "NV": {"lat":38.3135,"lon":-117.0554}, "NH": {"lat":43.4525,"lon":-71.5639},
    "NJ": {"lat":40.2989,"lon":-74.5210}, "NM": {"lat":34.8405,"lon":-106.2485},
    "NY": {"lat":42.1657,"lon":-74.9481}, "NC": {"lat":35.6301,"lon":-79.8064},
    "ND": {"lat":47.5289,"lon":-99.7840}, "OH": {"lat":40.3888,"lon":-82.7649},
    "OK": {"lat":35.5653,"lon":-96.9289}, "OR": {"lat":44.5720,"lon":-122.0709},
    "PA": {"lat":40.5908,"lon":-77.2098}, "RI": {"lat":41.6809,"lon":-71.5118},
    "SC": {"lat":33.8569,"lon":-80.9450}, "SD": {"lat":44.2998,"lon":-99.4388},
    "TN": {"lat":35.7478,"lon":-86.6923}, "TX": {"lat":31.0545,"lon":-97.5635},
    "UT": {"lat":40.1500,"lon":-111.8624}, "VT": {"lat":44.0459,"lon":-72.7107},
    "VA": {"lat":37.7693,"lon":-78.1699}, "WA": {"lat":47.4009,"lon":-121.4905},
    "WV": {"lat":38.4912,"lon":-80.9545}, "WI": {"lat":44.2685,"lon":-89.6165},
    "WY": {"lat":42.7560,"lon":-107.3025}
}

# ------------------------------------------------------------------
# 4) Prepare year & state lists
# ------------------------------------------------------------------
years = sorted(agg['death_year'].unique())
states = ['All'] + sorted(agg['state_abbr'].unique())

# ------------------------------------------------------------------
# 5) Build the figure
# ------------------------------------------------------------------
fig = go.Figure()
for i, yr in enumerate(years):
    dff = agg[agg['death_year']==yr]
    # Choropleth
    fig.add_trace(go.Choropleth(
        locations    = dff['state_abbr'],
        locationmode = 'USA-states',
        z            = dff['death_count'],
        zmin         = agg['death_count'].min(),
        zmax         = agg['death_count'].max(),
        colorscale   = 'Reds',
        colorbar     = dict(title='Deaths'),
        customdata   = dff[['total_pop_15to64','total_incarceration_rate']].values,
        hovertemplate=(
            "<b>%{location}</b><br>"
            "Deaths: %{z}<br>"
            "Pop15–64: %{customdata[0]:,}<br>"
            "Incarc Rate: %{customdata[1]:.1f}<extra></extra>"
        ),
        visible      = (i==0)
    ))
    # Labels
    fig.add_trace(go.Scattergeo(
        lon         = [state_centroids[s]['lon'] for s in dff['state_abbr']],
        lat         = [state_centroids[s]['lat'] for s in dff['state_abbr']],
        text        = dff['state_abbr'],
        mode        = 'text',
        showlegend  = False,
        hoverinfo   = 'none',
        textfont    = dict(size=9, color='black'),
        visible     = (i==0)
    ))

# ------------------------------------------------------------------
# 6) Create dropdowns
# ------------------------------------------------------------------
# Year menu
year_buttons = []
for i, yr in enumerate(years):
    vis = [False]*(2*len(years))
    vis[2*i] = True; vis[2*i+1] = True
    year_buttons.append(dict(label=str(yr), method='update',
                             args=[{'visible':vis},
                                   {'title':f'US Prisoner Deaths — {yr}'}]))
# State menu
chor_indices = list(range(0, 2*len(years), 2))
state_buttons = []
for st in states:
    # build new z arrays per trace
    z_lists = []
    for yr in years:
        base = agg[agg['death_year']==yr]
        z_lists.append([ (base.loc[base['state_abbr']==st,'death_count'].values[0] 
                          if st!='All' and st in base['state_abbr'].values else
                          (base['death_count'] if st=='All' else 0)).tolist()
                         for _ in base['state_abbr']])
    state_buttons.append(dict(label=st, method='restyle',
                              args=[{'z': z_lists}, chor_indices]))

# ------------------------------------------------------------------
# 7) Update layout
# ------------------------------------------------------------------
fig.update_layout(
    title=f'US Prisoner Deaths — {years[0]}',
    geo=dict(scope='usa', showlakes=True, lakecolor='white', bgcolor='#F0F0F0'),
    updatemenus=[
        dict(x=0.05, y=1.05, direction='down', buttons=year_buttons),
        dict(x=0.05, y=0.95, direction='down', buttons=state_buttons)
    ],
    margin=dict(l=0, r=0, t=60, b=0)
)

# ------------------------------------------------------------------
# 8) Show / Save
# ------------------------------------------------------------------
fig.write_html('prisoner_deaths_map_with_filters.html', include_plotlyjs='cdn')
fig.show()
